In [1]:
sql = '''
-- Ежедневный  показатель  
WITH Temp_2 AS (
    SELECT deals.*, users.Group
    FROM mt5r1.mt5_deals AS deals
    JOIN mt5r1.mt5_users AS users ON deals.Login = users.Login
    WHERE SUBSTRING(users.Group, 1, 12) = 'real\\\\neobit\\\\' and deals.Symbol <> '' and date(`Time`) < CURDATE()
)
SELECT 
date(`Time`) as Date,
Symbol,
sum(ContractSize * Volume),
sum(ContractSize * Volume * Price),
count(ContractSize)
FROM Temp_2
group by date(`Time`),Symbol
;
'''
CONFIG_1 = {
    'user': 'neo_reports',
    'password': 'gh2uyti56hgk2h',
    'host': '92.38.186.22',
    'port': '3306',
    'raise_on_warnings': True
}

CONFIG_2 = {
    'user': 'tester',
    'password': '250807',
    'host': '192.168.1.6',
    'port': '3306',
    'raise_on_warnings': True
}

In [2]:
import mysql.connector
import pandas as pd

In [5]:
sql_2 = '''
WITH Temp_2 AS (
    SELECT deals.*, users.`Group`
    FROM mt5r1.mt5_deals AS deals
    JOIN mt5r1.mt5_users AS users ON deals.Login = users.Login
    WHERE SUBSTRING(users.`Group`, 1, 12) = 'real\\\\neobit\\\\'
)

SELECT *
FROM Temp_2 t1
INNER JOIN Temp_2 t2 ON t1.TimeMsc >= TIMESTAMPADD(MICROSECOND, - 1500000, t2.TimeMsc)
                      AND t1.TimeMsc <= TIMESTAMPADD(MICROSECOND, 1500000, t2.TimeMsc)
                      AND t1.Entry != t2.Entry
                      AND t1.Symbol = t2.Symbol
WHERE t1.Login != t2.Login
--   AND ((t1.Entry = 0 AND t2.Entry = 1) OR (t1.Entry = 1 AND t2.Entry = 0))
limit 100
;
'''

In [12]:
connection = mysql.connector.connect(**CONFIG_1)
# Создать объект cursor для выполнения SQL-запросов
cursor = connection.cursor()
# cursor.execute(SQL_1)
# Получить результаты запроса
# results1 = cursor.fetchall()
df = pd.read_sql(sql_2, connection)
# cursor.execute(SQL_2)
# # Получить результаты запроса
# results2 = cursor.fetchall()
cursor.close()
connection.close()
df.head()

C:\Temp\ipykernel_12784\4169768486.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_2, connection)


Deal           Timestamp ExternalID   Login  Dealer  Order  Action  Entry  \
0  6046  133416697428575870       1596  320038       1   6187       0      0   
1  6043  133416697422801316       1595  320104       1   6184       0      1   
2  6109  133416702259603772       1618  320104       1   6254       0      0   
3  6106  133416702254068578       1617  320095       1   6251       1      1   
4  7315  133419889474434136       2032  320090       1   7470       1      1   

   Reason  Digits  ...    Gateway  PriceGateway ModifyFlags MarketBid  \
0      16       5  ...  MT5C24GWT       1.05196           0   1.05187   
1       3       5  ...  MT5C24GWT       1.05196           0   1.05187   
2      16       5  ...  MT5C24GWT       1.05235           0   1.05236   
3      16       5  ...  MT5C24GWT       1.05245           0   1.05237   
4      16       5  ...  MT5C24GWT       1.05499           0   1.05490   

  MarketAsk  MarketLast  Volume  VolumeClosed  ApiData  \
0   1.05196         0.0     500           500       []   
1   1.05196         0.0     100             0       []   
2   1.05245         0.0    1000          1000       []   
3   1.05246         0.0    1000             0       []   
4   1.05499         0.0    2000             0       []   

                         Group  
0  real\neobit\bbook_hedge_USD  
1  real\neobit\bbook_hedge_USD  
2  real\neobit\bbook_hedge_USD  
3  real\neobit\bbook_hedge_USD  
4  real\neobit\bbook_hedge_USD  

[5 rows x 88 columns]

In [13]:
df = df.drop_duplicates()
df.head()

Deal           Timestamp ExternalID   Login  Dealer  Order  Action  Entry  \
0  6046  133416697428575870       1596  320038       1   6187       0      0   
1  6043  133416697422801316       1595  320104       1   6184       0      1   
2  6109  133416702259603772       1618  320104       1   6254       0      0   
3  6106  133416702254068578       1617  320095       1   6251       1      1   
4  7315  133419889474434136       2032  320090       1   7470       1      1   

   Reason  Digits  ...    Gateway  PriceGateway ModifyFlags MarketBid  \
0      16       5  ...  MT5C24GWT       1.05196           0   1.05187   
1       3       5  ...  MT5C24GWT       1.05196           0   1.05187   
2      16       5  ...  MT5C24GWT       1.05235           0   1.05236   
3      16       5  ...  MT5C24GWT       1.05245           0   1.05237   
4      16       5  ...  MT5C24GWT       1.05499           0   1.05490   

  MarketAsk  MarketLast  Volume  VolumeClosed  ApiData  \
0   1.05196         0.0     500           500       []   
1   1.05196         0.0     100             0       []   
2   1.05245         0.0    1000          1000       []   
3   1.05246         0.0    1000             0       []   
4   1.05499         0.0    2000             0       []   

                         Group  
0  real\neobit\bbook_hedge_USD  
1  real\neobit\bbook_hedge_USD  
2  real\neobit\bbook_hedge_USD  
3  real\neobit\bbook_hedge_USD  
4  real\neobit\bbook_hedge_USD  

[5 rows x 88 columns]

In [17]:
df['TimeMsc'].tail()

TimeMsc                 TimeMsc
95 2023-11-10 18:00:00.745 2023-11-10 18:00:00.849
96 2023-11-14 07:16:48.123 2023-11-14 07:16:48.037
97 2023-11-14 07:16:48.037 2023-11-14 07:16:48.123
98 2023-11-14 08:05:00.842 2023-11-14 08:05:00.741
99 2023-11-14 08:05:00.741 2023-11-14 08:05:00.842

In [18]:
sql_3 = '''
WITH Temp_2 AS (
    SELECT deals.*, users.`Group`
    FROM mt5r1.mt5_deals AS deals
    JOIN mt5r1.mt5_users AS users ON deals.Login = users.Login
    WHERE SUBSTRING(users.`Group`, 1, 12) = 'real\\\\neobit\\\\' and `Time` >= "2023-11-14"
)

SELECT *
FROM Temp_2 t1
INNER JOIN Temp_2 t2 ON t1.TimeMsc >= TIMESTAMPADD(MICROSECOND, - 1500000, t2.TimeMsc)
                      AND t1.TimeMsc <= TIMESTAMPADD(MICROSECOND, 1500000, t2.TimeMsc)
                      AND t1.Entry != t2.Entry
                      AND t1.Symbol = t2.Symbol
WHERE t1.Login != t2.Login
--   AND ((t1.Entry = 0 AND t2.Entry = 1) OR (t1.Entry = 1 AND t2.Entry = 0))
limit 100
;'''

In [19]:
connection = mysql.connector.connect(**CONFIG_1)
# Создать объект cursor для выполнения SQL-запросов
cursor = connection.cursor()
# cursor.execute(SQL_1)
# Получить результаты запроса
# results1 = cursor.fetchall()
df_2 = pd.read_sql(sql_3, connection)
# cursor.execute(SQL_2)
# # Получить результаты запроса
# results2 = cursor.fetchall()
cursor.close()
connection.close()
df_2.head()

C:\Temp\ipykernel_12784\3488818580.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_2 = pd.read_sql(sql_3, connection)


Deal           Timestamp ExternalID   Login  Dealer  Order  Action  Entry  \
0  25456  133444090081236546       7953  320097       1  25915       0      0   
1  25453  133444090080378841       7952  320133       1  25914       1      1   
2  25513  133444119008426103       7989  320098       1  25990       0      0   
3  25510  133444119007418797       7988  320218       1  25989       1      1   
4  26409  133444500177868254       8276  320044       1  26883       1      1   

   Reason  Digits  ...    Gateway  PriceGateway ModifyFlags MarketBid  \
0      16       5  ...  MT5C24GWT       1.06963           0   1.06964   
1      16       5  ...  MT5C24GWT       1.06973           0   1.06964   
2      16       5  ...  MT5C24GWT       1.06950           0   1.06951   
3       3       5  ...  MT5C24GWT       1.06959           0   1.06951   
4      16       5  ...  MT5C24GWT       1.08342           0   1.08343   

  MarketAsk  MarketLast  Volume  VolumeClosed  ApiData  \
0   1.06973         0.0    1000          1000       []   
1   1.06973         0.0     100             0       []   
2   1.06960         0.0     100           100       []   
3   1.06960         0.0     100             0       []   
4   1.08352         0.0     300             0       []   

                         Group  
0  real\neobit\bbook_hedge_USD  
1  real\neobit\bbook_hedge_USD  
2  real\neobit\bbook_hedge_USD  
3  real\neobit\bbook_hedge_USD  
4  real\neobit\bbook_hedge_USD  

[5 rows x 88 columns]

In [22]:
df_2 = df_2.drop_duplicates()

In [26]:
sql_3 = '''
WITH Temp_2 AS (
    SELECT deals.*, users.`Group`
    FROM mt5r1.mt5_deals AS deals
    JOIN mt5r1.mt5_users AS users ON deals.Login = users.Login
    WHERE SUBSTRING(users.`Group`, 1, 12) = 'real\\\\neobit\\\\' and `Time` >= "2023-11-28"
)

SELECT *
FROM Temp_2 t1
INNER JOIN Temp_2 t2 ON t1.TimeMsc >= TIMESTAMPADD(MICROSECOND, - 1500000, t2.TimeMsc)
                      AND t1.TimeMsc <= TIMESTAMPADD(MICROSECOND, 1500000, t2.TimeMsc)
                      AND t1.Entry != t2.Entry
                      AND t1.Symbol = t2.Symbol
WHERE t1.Login != t2.Login
--   AND ((t1.Entry = 0 AND t2.Entry = 1) OR (t1.Entry = 1 AND t2.Entry = 0))
-- limit 100
; '''

In [27]:
connection = mysql.connector.connect(**CONFIG_1)
# Создать объект cursor для выполнения SQL-запросов
cursor = connection.cursor()
# cursor.execute(SQL_1)
# Получить результаты запроса
# results1 = cursor.fetchall()
df_3 = pd.read_sql(sql_3, connection)
# cursor.execute(SQL_2)
# # Получить результаты запроса
# results2 = cursor.fetchall()
cursor.close()
connection.close()
df_3 = df_3.drop_duplicates()

C:\Temp\ipykernel_12784\3283830794.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_3 = pd.read_sql(sql_3, connection)


In [29]:
# Список ваших DataFrame
dataframes = [df, df_2, df_3]

# Объединение DataFrame вдоль строк (в конце одного начинается другой)
combined_df = pd.concat(dataframes, ignore_index=True, axis=0)
combined_df.iloc[90:120]

Deal           Timestamp ExternalID   Login  Dealer  Order  Action  \
90   24260  133440660370461783       7562  320159       1  24685       0   
91   24257  133440660370022956       7561  320205       1  24684       0   
92   24666  133441020007453877       7690  320261       1  25083       1   
93   24669  133441020008491103       7691  320141       1  25090       1   
94   24648  133441020004771052       7684  320201       1  25077       0   
95   24666  133441020007453877       7690  320261       1  25083       1   
96   25456  133444090081236546       7953  320097       1  25915       0   
97   25453  133444090080378841       7952  320133       1  25914       1   
98   25513  133444119008426103       7989  320098       1  25990       0   
99   25510  133444119007418797       7988  320218       1  25989       1   
100  25456  133444090081236546       7953  320097       1  25915       0   
101  25453  133444090080378841       7952  320133       1  25914       1   
102  25513  133444119008426103       7989  320098       1  25990       0   
103  25510  133444119007418797       7988  320218       1  25989       1   
104  26409  133444500177868254       8276  320044       1  26883       1   
105  26406  133444500165062504       8275  320098       1  26880       1   
106  26983  133444686030932651       8469  320267       1  27467       1   
107  26980  133444686027072974       8468  320267       1  27462       0   
108  26977  133444686026873507       8467  320213       1  27461       1   
109  26977  133444686026873507       8467  320213       1  27461       1   
110  27242  133445237636760898       8556  320267       1  27722       1   
111  27239  133445237633210395       8555  320267       1  27717       0   
112  27236  133445237633040848       8554  320213       1  27716       1   
113  27236  133445237633040848       8554  320213       1  27716       1   
114  28144  133446006645279057       8850  320267       1  28636       1   
115  28141  133446006641489194       8849  320267       1  28631       0   
116  28138  133446006641379487       8848  320213       1  28630       1   
117  28138  133446006641379487       8848  320213       1  28630       1   
118  29480  133446907664755670       9257  320213       1  29976       1   
119  29483  133446907665214442       9258  320267       1  29979       1   

     Entry  Reason  Digits  ...    Gateway  PriceGateway ModifyFlags  \
90       1      16       5  ...  MT5C24GWT       1.06717           0   
91       1      16       5  ...  MT5C24GWT       1.06717           0   
92       0       1       5  ...  MT5C24GWT       1.21956           0   
93       1       3       5  ...  MT5C24GWT       1.21896           0   
94       1       4       5  ...  MT5C24GWT       1.21896           0   
95       0       1       5  ...  MT5C24GWT       1.21895           0   
96       0      16       5  ...  MT5C24GWT       1.06963           0   
97       1      16       5  ...  MT5C24GWT       1.06973           0   
98       0      16       5  ...  MT5C24GWT       1.06950           0   
99       1       3       5  ...  MT5C24GWT       1.06959           0   
100      0      16       5  ...  MT5C24GWT       1.06963           0   
101      1      16       5  ...  MT5C24GWT       1.06973           0   
102      0      16       5  ...  MT5C24GWT       1.06950           0   
103      1       3       5  ...  MT5C24GWT       1.06959           0   
104      1      16       5  ...  MT5C24GWT       1.08342           0   
105      0      16       5  ...  MT5C24GWT       1.08342           0   
106      0       1       5  ...  MT5C24GWT       0.65066           0   
107      0       1       5  ...  MT5C24GWT       0.65066           0   
108      1       3       5  ...  MT5C24GWT       0.65083           0   
109      1       3       5  ...  MT5C24GWT       0.65067           0   
110      0       1       5  ...  MT5C24GWT       0.65257           0   
111      0       1       5  ...  MT5C24GWT       0.65257         

In [32]:
combined_df = combined_df.drop_duplicates()
combined_df.to_excel('Подозрительные Джентельмены.xlsx', index=False)

In [34]:
from datetime import datetime, timedelta

current_time = datetime.now()
# Прибавление 3 часов к текущему времени
time_after_3_hours = current_time + timedelta(hours= 3 + 24)
# Форматирование времени до минут в формате "ггггммддд чч:мм"
future_date = time_after_3_hours.strftime("%Y-%m-%d")
future_date

'2023-12-12'